In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
X = pd.read_csv(os.path.join(dirname, 'train.csv'))
X_Test = pd.read_csv(os.path.join(dirname, 'test.csv'))

#############
oil = pd.read_csv(os.path.join(dirname, 'oil.csv'))
oil['dcoilwtico'].fillna(oil['dcoilwtico'].mode()[0], inplace=True)
############


X
# oil

#  Convert family names to numbers and dates to oil prizes

In [ ]:
import time
from tqdm import tqdm, notebook, trange

In [ ]:
def familyToNumReplace_and_oil(df, df_oil, family_names, family_names_num):
    family_name_to_num=[]; oil_prices=[]
    for i in trange(df.shape[0]):
        pos1 = np.where(family_names == df.family[i])
        family_name_to_num.append(family_names_num[pos1[0][0]])
        
        pos2 = np.where(df_oil.date == df.date[i])
        if(len(pos2[0])>0):
            oil_prices.append(df_oil.dcoilwtico[pos2[0][0]])
        if(len(pos2[0])==0):
            oil_prices.append(oil['dcoilwtico'].mode()[0])
        
    return np.asarray(family_name_to_num), np.asarray(oil_prices)


In [ ]:
family_names = X.family.drop_duplicates()
family_names_num = np.arange(len(family_names))

###### family names to numbers
%time family_name_to_num_train, oil_prices_train = familyToNumReplace_and_oil(X, oil, family_names, family_names_num)
X['family_num'] = family_name_to_num_train
X['oil_price'] = oil_prices_train

%time family_name_to_num_test, oil_prices_test = familyToNumReplace_and_oil(X_Test, oil, family_names, family_names_num)
X_Test['family_num'] = family_name_to_num_test
X_Test['oil_price'] = oil_prices_test


X

In [ ]:
X_Test

In [ ]:
X.columns

In [ ]:
from sklearn.model_selection import train_test_split


features = ['store_nbr', 'onpromotion','family_num', 'oil_price']

X_Dat = X[features]; Y_Dat = X['sales']
X_TestN = X_Test[features];

train_X, val_X, train_y, val_y = train_test_split(X_Dat, Y_Dat, random_state=0)

In [ ]:
train_X

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_log_error

#### Define the model. Set random_state to 1
rf_model = RandomForestRegressor(random_state=1)

#### fit your model
rf_model.fit(train_X, train_y)

trainfit_y = rf_model.predict(train_X)
valfit_y = rf_model.predict(val_X)

#### Calculate the mean absolute error of your Random Forest model on the validation data
train_mae = mean_squared_log_error(trainfit_y, train_y)
rf_val_mae = mean_squared_log_error(valfit_y, val_y)

print("Train MSE: {}".format(train_mae))
print("Validation MAE: {}".format(rf_val_mae))

In [ ]:
import matplotlib.pyplot as plt
fig_size = plt.rcParams["figure.figsize"]
fig_size[0] = 10
fig_size[1] = 3

transient=2000

xval = np.arange(0,len(train_y[::transient]))
plt.plot(xval,train_y[::transient])
plt.plot(xval,trainfit_y[::transient])
plt.xlim(0,100)
plt.ylim(0,5000)
plt.show()

xval = np.arange(0,len(val_y[::transient]))
plt.plot(xval,val_y[::transient])
plt.plot(xval,valfit_y[::transient])
plt.xlim(0,100)
plt.ylim(0,5000)
plt.show()


In [ ]:
Predictions_Rf = rf_model.predict(X_TestN)

plt.plot(Predictions_Rf)
# plt.xlim(0,300)
plt.show()

# Submission 

In [ ]:
X_Test['sales'] = Predictions_Rf

Prediction_table = X_Test[['id', 'sales']].copy()
Prediction_table

In [ ]:
# Prediction_table.to_csv(os.path.join(dirname,'submission.csv'), index=False)

sub = pd.read_csv('../input/store-sales-time-series-forecasting/sample_submission.csv')
sub['sales'] = Predictions_Rf
sub.to_csv('submission.csv', index = False) # Submit
sub